# Proyek Analisis Data: Beijing Air Quality (PRSA 2013-2017)
- **Nama:** Mohamamd 'Alwan Fauzi
- **Email:** alwanfauzi111@gmail.com
- **ID Dicoding:** alwanfauzi

**Definisi PM2.5:**
PM2.5 adalah konsentrasi partikel udara berukuran sangat kecil (diameter 2,5 mikrometer) yang tersuspensi di udara.
Di dataset ini, PM2.5 direkam sebagai kadar partikel per jam di tiap stasiun pengukuran.

## Menentukan Pertanyaan Bisnis

- Stasiun mana yang memiliki rata-rata PM2.5 tertinggi pada periode 2013-2017, dan bagaimana perbandingan antar stasiun?
- Bagaimana pola musiman PM2.5 pada periode 2013-2017 (rata-rata bulanan)?
- Bagaimana perbandingan rata-rata polutan lain (PM10, NO2, O3) antar stasiun pada periode 2013-2017?


## Import Semua Packages/Library yang Digunakan

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

sns.set_theme(style='whitegrid')

## Data Wrangling

### Gathering Data

In [ ]:
data_dir = Path("dataset")
files = sorted(data_dir.glob("*.csv"))
dfs = [pd.read_csv(f) for f in files]
raw_df = pd.concat(dfs, ignore_index=True)
raw_df.shape

**Insight (objek: dataset PRSA 12 stasiun):**
- Objek yang dianalisis adalah 12 file dataset stasiun kualitas udara dengan skema kolom yang konsisten.
- Total baris data gabungan berjumlah 420.768 sebelum pembersihan.

### Assessing Data

In [ ]:
raw_df.info()
raw_df.isna().sum().sort_values(ascending=False).head(10)
raw_df.duplicated().sum()

**Insight (objek: missing values pada kolom polutan):**
- Objek yang paling banyak missing adalah CO (20.701), O3 (13.277), NO2 (12.116), dan SO2 (9.021).
- Objek duplikasi baris pada data mentah tidak ditemukan.

### Cleaning Data

In [ ]:
df = raw_df.copy()
numeric_cols = ["PM2.5", "PM10", "SO2", "NO2", "CO", "O3", "TEMP", "PRES", "DEWP", "RAIN", "WSPM"]
for col in numeric_cols:
    df[col] = pd.to_numeric(df[col], errors="coerce")

df["wd"] = df["wd"].fillna("Unknown")
df["RAIN"] = df["RAIN"].fillna(0)
df["datetime"] = pd.to_datetime(df[["year", "month", "day", "hour"]])

df = df.dropna(subset=["PM2.5", "PM10", "SO2", "NO2", "CO", "O3", "TEMP", "PRES", "DEWP", "WSPM"])
df = df.drop_duplicates()
df.head()

**Insight (objek: dataset hasil cleaning):**
- Setelah pembersihan, objek data yang dianalisis menyisakan 383.589 baris.
- Objek waktu dibentuk sebagai kolom `datetime`, dan objek curah hujan kosong diisi 0.

## Exploratory Data Analysis (EDA)

### Explore ...

In [ ]:
df.describe()

pm25_by_station = df.groupby("station")["PM2.5"].mean().sort_values(ascending=False)
pm25_by_station

**Insight (objek: rata-rata PM2.5 per stasiun):**
- Rata-rata PM2.5 pada dataset ini adalah nilai mean konsentrasi PM2.5 per jam untuk setiap stasiun sepanjang 2013-2017 setelah pembersihan data.
- Objek stasiun dengan rata-rata PM2.5 tertinggi adalah Dongsi (84,98), Nongzhanguan (84,75), dan Wanshouxigong (84,27).
- Objek stasiun dengan rata-rata PM2.5 terendah adalah Dingling (66,62), Changping (70,42), dan Huairou (70,44).

## Visualization & Explanatory Analysis

### Pertanyaan 1: Stasiun mana yang memiliki rata-rata PM2.5 tertinggi pada periode 2013-2017?


In [ ]:
plt.figure(figsize=(10, 5))
pm25_by_station.plot(kind="bar", color="teal")
plt.title("Rata-rata PM2.5 per Stasiun")
plt.ylabel("PM2.5")
plt.xlabel("Stasiun")
plt.xticks(rotation=45, ha="right")
plt.tight_layout()

### Pertanyaan 2: Bagaimana pola musiman PM2.5 pada periode 2013-2017?


In [ ]:
df["month"] = df["datetime"].dt.month
monthly_pm25 = df.groupby("month")["PM2.5"].mean()

plt.figure(figsize=(8, 4))
monthly_pm25.plot(marker="o")
plt.title("Pola Bulanan Rata-rata PM2.5 (Semua Stasiun)")
plt.xlabel("Bulan")
plt.ylabel("PM2.5")
plt.xticks(range(1, 13))
plt.tight_layout()

**Insight (objek: kategori polusi dan pola bulanan PM2.5):**
- Objek stasiun dengan proporsi kategori polusi berat tertinggi adalah Dongsi, Wanshouxigong, dan Gucheng.
- Objek bulan dengan rata-rata PM2.5 tertinggi adalah Desember (105,72) dan terendah adalah Agustus (53,31).

### Pertanyaan 3: Bagaimana perbandingan rata-rata polutan lain (PM10, NO2, O3) antar stasiun pada periode 2013-2017?


In [ ]:
pollutants = ["PM10", "NO2", "O3"]
mean_pollutants = df.groupby("station")[pollutants].mean()

plt.figure(figsize=(8, 5))
sns.heatmap(mean_pollutants, cmap="YlGnBu")
plt.title("Rata-rata PM10, NO2, O3 per Stasiun (2013-2017)")
plt.xlabel("Polutan")
plt.ylabel("Stasiun")
plt.tight_layout()


**Insight (Pertanyaan 3 -- ringkas):**
- **PM10 tertinggi:** Gucheng (119,15), Wanshouxigong (111,93), Dongsi (111,05).
- **NO2 tertinggi:** Wanliu (64,47), Aotizhongxin (59,02), Nongzhanguan (58,25).
- **O3 tertinggi:** Dingling (69,33), Huairou (59,45), Nongzhanguan (58,54).
- **Inti:** stasiun dominan berbeda untuk tiap polutan.

**Insight (objek: perbandingan PM10, NO2, O3 per stasiun):**
- Heatmap memperlihatkan variasi konsentrasi rata-rata antar stasiun untuk tiap polutan.
- Warna lebih gelap menunjukkan stasiun dengan nilai rata-rata lebih tinggi pada polutan terkait, sehingga mudah mengidentifikasi stasiun prioritas pemantauan.


## Analisis Lanjutan (Opsional)

In [ ]:
def month_to_season(month):
    if month in [12, 1, 2]:
        return "DJF"
    if month in [3, 4, 5]:
        return "MAM"
    if month in [6, 7, 8]:
        return "JJA"
    return "SON"

df["season"] = df["month"].map(month_to_season)
bins = [0, 35, 75, 115, 150, 250, 500]
labels = ["Good", "Moderate", "Unhealthy(SG)", "Unhealthy", "Very Unhealthy", "Hazardous"]
df["pm25_level"] = pd.cut(df["PM2.5"], bins=bins, labels=labels, right=False)

season_dist = (
    df.dropna(subset=["pm25_level"])
      .groupby(["season", "pm25_level"])
      .size()
      .groupby(level=0)
      .apply(lambda x: x / x.sum())
      .unstack()
)
season_dist

season_dist.plot(kind="bar", stacked=True, figsize=(8, 4), colormap="viridis")
plt.title("Proporsi Kategori PM2.5 per Musim")
plt.xlabel("Musim")
plt.ylabel("Proporsi")
plt.legend(title="Kategori PM2.5", bbox_to_anchor=(1.02, 1), loc="upper left")
plt.tight_layout()

## Menyimpan Dataset untuk Dashboard

In [ ]:
from pathlib import Path

dashboard_dir = Path("dashboard")
dashboard_dir.mkdir(exist_ok=True)

cols = ["datetime", "year", "month", "season", "station",
        "PM2.5", "PM10", "SO2", "NO2", "CO", "O3",
        "TEMP", "PRES", "DEWP", "RAIN", "wd", "WSPM"]

df[cols].to_csv(dashboard_dir / "main_data.csv", index=False)
df[cols].head()

## Conclusion

- Stasiun prioritas PM2.5: Dongsi, Nongzhanguan, Wanshouxigong (rata-rata PM2.5 tertinggi), dengan Dongsi paling sering berada di kategori polusi berat.
- Waktu prioritas mitigasi: Desember (puncak PM2.5) dan Agustus (terendah), dengan variasi pola antar stasiun.
- Polutan lain menunjukkan dominasi berbeda: PM10 tertinggi di Gucheng, NO2 di Wanliu, dan O3 di Dingling (tidak selalu sama stasiunnya).